In [54]:
from webull import paper_webull,webull
from dotenv import load_dotenv
import os
import time
from datetime import date
import math
import requests

In [55]:
def loginWebull(paper = True):
    # Determine paper vs real trading
    if paper:
        wb = paper_webull()
    else:
        wb = webull()

    # Get environment variables and log in
    wb._set_did = os.getenv("DID")
    wb._access_token = os.getenv("ACCESS_TOKEN")
    wb.uuid = os.getenv("UUID")
    login_result = wb.login(os.getenv("LOGIN"),os.getenv("PASSWORD"), "Jaydon's Laptop", save_token=True)

    # Check that log in was successful
    if wb.is_logged_in():
        account_id = wb.get_account_id()
        account_type = wb.get_account_type(os.getenv("LOGIN"))
        print('-----------------------------------')
        print('>>>>>>   Log in successful   <<<<<<')
        print(f'>>>   Your login ID: {account_id}   <<<')
        print(f'>>>   Your account type ID: {account_type}   <<<')
    else:
        print('-----------------------------------')
        print('>>> Log in failed, authentication failed, check info below: ')
        print(login_result)
        return None

    # Attempt to enable trading
    print('-----------------------------------')
    print('>>> Enable trading requested <<<')
    if wb.get_trade_token(os.getenv("PID")):
        print('Trading enabled, authentication passed')
    else:
        if wb.is_logged_in():
            print('Authentication failed, check PID again')
        else:
            print('Authentication failed, please log in again')
        return None
    return wb


In [56]:
load_dotenv()
wb = loginWebull()
print(wb.get_portfolio())
print(wb.get_positions())


-----------------------------------
>>>>>>   Log in successful   <<<<<<
>>>   Your login ID: 24520358   <<<
>>>   Your account type ID: 2   <<<
-----------------------------------
>>> Enable trading requested <<<
Trading enabled, authentication passed
{'totalMarketValue': '78.00', 'usableCash': '3906.34', 'dayProfitLoss': '-1298.06'}
[{'id': 501589381, 'accountId': 24520358, 'paperId': 1, 'ticker': {'tickerId': 1051141280, 'exchangeId': 189, 'type': 7, 'regionId': 6, 'regionCode': 'US', 'currencyId': 247, 'symbol': 'SPY250320C00567000', 'disSymbol': 'SPY250320C00567000', 'disExchangeCode': 'OPRA', 'exchangeCode': 'OPRA', 'listStatus': 0, 'subType': 901, 'optionSymbol': 'SPY', 'belongTickerId': 913243251}, 'status': 1, 'position': '4', 'cost': '66', 'costPrice': '0.1650', 'lastPrice': '0.0200', 'marketValue': '8.00', 'marketValueNoScale': 8.0, 'unrealizedProfitLoss': '-58.00', 'unrealizedProfitLossRate': '-0.8788', 'tickerType': 'OPTION', 'optionType': 'call', 'optionExpireDate': '2025-

In [57]:
print(wb.place_order(stock='SPY', action='BUY', orderType='MKT', enforce='DAY', quant=1, outsideRegularTradingHour=True))
time.sleep(10)
print(wb.get_portfolio())
print(wb.get_positions())

{'orderId': 202300171}
{'totalMarketValue': '78.00', 'usableCash': '3285.98', 'dayProfitLoss': '-1298.06'}
[{'id': 501589381, 'accountId': 24520358, 'paperId': 1, 'ticker': {'tickerId': 1051141280, 'exchangeId': 189, 'type': 7, 'regionId': 6, 'regionCode': 'US', 'currencyId': 247, 'symbol': 'SPY250320C00567000', 'disSymbol': 'SPY250320C00567000', 'disExchangeCode': 'OPRA', 'exchangeCode': 'OPRA', 'listStatus': 0, 'subType': 901, 'optionSymbol': 'SPY', 'belongTickerId': 913243251}, 'status': 1, 'position': '4', 'cost': '66', 'costPrice': '0.1650', 'lastPrice': '0.0200', 'marketValue': '8.00', 'marketValueNoScale': 8.0, 'unrealizedProfitLoss': '-58.00', 'unrealizedProfitLossRate': '-0.8788', 'tickerType': 'OPTION', 'optionType': 'call', 'optionExpireDate': '2025-03-20', 'optionContractMultiplier': 100.0, 'optionExercisePrice': 567.0, 'belongTickerId': 913243251}, {'id': 501567082, 'accountId': 24520358, 'paperId': 1, 'ticker': {'tickerId': 1051182302, 'exchangeId': 189, 'type': 7, 'regio

In [58]:
def paper_place_order_option(wb, optionId=None, lmtPrice=None, stpPrice= None, action=None, orderType='LMT', enforce='DAY', quant=0 ):
    '''
        create buy / sell order
        optionId: int
        lmtPrice: float
        stpPrice: float
        action: string BUY / SELL
        orderType: MKT / LMT / STP / STP LMT
        enforce: GTC / DAY
        quant: int
    '''

    # Paper trading URL
    url = "https://act.webullfintech.com/webull-paper-center/api/paper/v1/order/optionPlace"

    # Request headers
    headers = wb.build_req_headers(include_trade_token=True, include_time=True)

    # Account details
    account_id = wb._account_id

    # Request payload
    data = {
        "accountId": account_id,
        "orderType": orderType,
        "timeInForce": enforce,
        "quantity": quant,
        "action": action,
        "tickerId": optionId,
        "orders": [
            {
                "action": action,
                "quantity": quant,
                "tickerId": optionId,
                "tickerType": "OPTION"
            }
        ],
        "checkOrPlace": "PLACE",
        "paperId": 1,
        "tickerType": "OPTION",
        "optionStrategy": "Single",
    }
    if orderType == 'LMT' and lmtPrice :
        data['lmtPrice'] = float(lmtPrice)
    elif orderType == 'STP' and stpPrice :
        data['auxPrice'] = float(stpPrice)
    elif orderType == 'STP LMT' and lmtPrice and stpPrice :
        data['lmtPrice'] = float(lmtPrice)
        data['auxPrice'] = float(stpPrice)
        
    response = requests.post(url, json=data, headers=headers, timeout=wb.timeout)
    if response.status_code != 200:
        raise Exception('place_option_order failed', response.status_code, response.reason)
    return response.json()

In [59]:
# Calculate Option Strike to Buy (Just slight OTM)
stock_info = wb.get_quote("SPY")

best_bid_price = float(stock_info['bidList'][0]['price'])
best_bid_volume = float(stock_info['bidList'][0]['volume'])
best_ask_price = float(stock_info['askList'][0]['price'])
best_ask_volume = float(stock_info['askList'][0]['volume'])

current_price = (best_bid_price * best_bid_volume + best_ask_price * best_ask_volume) / (best_bid_volume + best_ask_volume)
strike_price = str(math.ceil(current_price))


In [60]:
# Get Date for Option to Buy (ODTE)
today = date.today()
formatted_date = today.strftime("%Y-%m-%d")  # Year-month-day

# Get Option Ticker for Given Strike
options = wb.get_options_by_strike_and_expire_date(stock='SPY', expireDate=formatted_date,strike=strike_price, direction='call')
tickerId = options[0]['call']['tickerId']

In [ ]:
print(paper_place_order_option(wb, optionId = tickerId, action='BUY',orderType='MKT', enforce='DAY', quant=1))

{'success': True, 'orderId': '202300184'}


In [75]:
print(wb.alerts_add("SPY"))

Exception: ('alerts_add failed', 417, '')